After having created a list of matched unique speakers by legislature from both the CLD and TWFY dataset, we can now merge the speaker_id information from the raw text contained in the scraped files with the additional metadata contained in the Comparative Legislators Database.

Again, we will use the example of parliamentary data from the 1970s. 

In [1]:
import pandas as pd
import numpy as np 

In [2]:
df = pd.read_pickle("C:/Users/jnjco/Desktop/Personal_UK_DE_inflation/Scraping_UK/Scraping_1970s/uk_debates_1970_1979.pkl")

In [5]:
df.columns = ["speech_id_link", "speaker", "twfy_member_id", "hansard_id", "text"]

In [6]:
df.head()

,speech_id_link,speaker,twfy_member_id,hansard_id,text
0,uk.org.publicwhip/debate/1970-01-19a.1.5,Sir Robert Cary,uk.org.publicwhip/member/13195,2177,"On a point of order. Is it true, Mr. Speaker, ..."
1,uk.org.publicwhip/debate/1970-01-19a.1.6,Mr. Speaker,uk.org.publicwhip/member/29799,10930,I am sorry to confirm what the hon. Gentleman ...
2,uk.org.publicwhip/debate/1970-01-19a.1.7,The Secretary of State for Wales (Mr. George T...,uk.org.publicwhip/member/31250,10844,On behalf of hon. and right hon. Members on th...
3,uk.org.publicwhip/debate/1970-01-19a.2.1,Mr. Speaker,uk.org.publicwhip/member/29799,10930,I have to report that I have received a letter...
4,uk.org.publicwhip/debate/1970-01-19a.2.4,Mr. Fred Evans,unknown,None,asked the Secretary of State for Wales whether...


In [7]:
df['date'] = df['speech_id_link'].str.extract(r'(\d{4}-\d{2}-\d{2})')


In [8]:
legislatures = [
    {"number": 45, "start": "1970-06-18", "end": "1974-02-27"},
    {"number": 46, "start": "1974-02-28", "end": "1974-10-09"},
    {"number": 47, "start": "1974-10-10", "end": "1979-05-02"},
    {"number": 48, "start": "1979-05-03", "end": "1983-06-08"},
    {"number": 49, "start": "1983-06-09", "end": "1987-06-10"},
    {"number": 50, "start": "1987-06-11", "end": "1992-04-08"},
    {"number": 51, "start": "1992-04-09", "end": "1997-04-30"},
    {"number": 52, "start": "1997-05-01", "end": "2001-06-06"},
    {"number": 53, "start": "2001-06-07", "end": "2005-05-04"},
    {"number": 54, "start": "2005-05-05", "end": "2010-05-05"},
    {"number": 55, "start": "2010-05-06", "end": "2015-05-06"},
    {"number": 56, "start": "2015-05-07", "end": "2017-06-07"},
    {"number": 57, "start": "2017-06-08", "end": "2019-12-11"},
    {"number": 58, "start": "2019-12-12", "end": "2024-05-22"}, 
]

In [9]:
for leg in legislatures:
    leg["start"] = pd.to_datetime(leg["start"])
    leg["end"] = pd.to_datetime(leg["end"])


In [10]:
df["date"] = pd.to_datetime(df["date"])


In [11]:
def get_legislature(date):
    for leg in legislatures:
        if leg["start"] <= date <= leg["end"]:
            return leg["number"]
    return None

df["legislature"] = df["date"].apply(get_legislature)


In [12]:
df = df[df["date"] > "1970-06-17"]

In [13]:
# Create a "chair" column to identify speakers who are chairs
df['chair'] = df['speaker'].str.contains(r'\bSpeaker\b', regex=True, na=False)

C:\Users\jnjco\AppData\Local\Temp\ipykernel_11044\1433088405.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['chair'] = df['speaker'].str.contains(r'\bSpeaker\b', regex=True, na=False)


In [ ]:
# Read Excel file with matched speaker IDs. Where the LLM was not able to matche a speaker, we manually matched them. 
# This means that all unique IDs are matched with a speaker name (and associated metadata) as recorded in the CLD dataset.
commonscorpus_id = pd.read_excel("./unique_speakers_integrated.xlsx")

In [ ]:
commonscorpus_id = commonscorpus_id.rename(columns={"member_id": "twfy_member_id"})

In [ ]:
commonscorpus_id["twfy_member_id"] = commonscorpus_id["twfy_member_id"].astype(str)

In [ ]:
df_new = pd.merge(df, commonscorpus_id, how="left", on = "twfy_member_id")

In [19]:
df_new = df_new.drop_duplicates(subset=["speech_id_link"], keep="first")

In [20]:
df_new = df_new.drop(columns=["Source.Name", "Column1", "speaker_y", "speaker_id", "hansard_id_y", "match_speaker", "check"])

In [21]:
mask_unknown = df_new['twfy_member_id'] == 'unknown'

# Fill 'date' and 'legislature' from the previous row for unknowns
df_new.loc[mask_unknown, 'date'] = df_new['date'].ffill()
df_new.loc[mask_unknown, 'legislature'] = df_new['legislature'].ffill()
df_new.loc[mask_unknown, 'session'] = df_new['session'].ffill()

# Set 'chair' to False
df_new.loc[mask_unknown, 'chair'] = False

# Columns to keep unchanged even if twfy_member_id is unknown
keep_columns = ['speech_id_link', 'speaker_x', 'text', 'date', 'legislature', 'session', 'chair']

# Columns to be set to NaN for "unknown" rows
cols_to_null = df_new.columns.difference(keep_columns)

# Set those to NaN
df_new.loc[mask_unknown, cols_to_null] = np.nan

In [22]:
df_new = df_new.rename(columns={"speaker_x": "speaker"}) 

In [23]:
df_new['chair'] = df_new['speaker'].str.contains(r'\bSpeaker\b', regex=True, na=False)

In [24]:
columns_to_keep = ['speech_id_link', 'speaker', 'text', 'date', 'legislature']

# Get all columns in the dataframe
all_columns = df_new.columns

# Find columns to nullify
columns_to_nullify = [col for col in all_columns if col not in columns_to_keep]

# Set specified columns to NaN where chair is True
df_new.loc[df_new['chair'] == True, columns_to_nullify] = np.nan

C:\Users\jnjco\AppData\Local\Temp\ipykernel_11044\2072126069.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df_new.loc[df_new['chair'] == True, columns_to_nullify] = np.nan
